# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [58]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key
print(geoapify_key)

9c5f72e38fb74e069324f55103729448


In [59]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()
city_data_df.shape

(566, 10)

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)

map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [67]:
# Narrow down cities that fit criteria and drop any results with null values

narrowed_city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 50) & (city_data_df["Max Temp"] > 15) \
    & (city_data_df["Wind Speed"] < 6.0) \
    & (city_data_df["Cloudiness"] == 0)
]

narrowed_city_data_df = narrowed_city_data_df.dropna()

# Display sample data
narrowed_city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,palmer,42.1584,-72.3287,44.62,37,0,5.75,US,1707079552
65,65,laojunmiao,39.8333,97.7333,17.31,50,0,1.66,CN,1707079710
119,119,zhangye,38.9342,100.4517,21.81,56,0,1.30,CN,1707079719
175,175,prince george,53.9166,-122.7530,26.62,73,0,5.75,CA,1707079729
525,525,fort mcmurray,56.7268,-111.3810,15.39,79,0,4.61,CA,1707079540
531,531,jinchang,38.4953,102.1739,18.18,62,0,4.14,CN,1707079791
557,557,chibougamau,49.9168,-74.3659,22.62,74,0,2.30,CA,1707079795


### Step 3: Create a new DataFrame called `hotel_df`.

In [68]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
#hotel_df = pd.DataFrame(narrowed_city_data_df, columns=["City", "Country", "Lat", "Lng", "Humidity"])
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = narrowed_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,palmer,US,42.1584,-72.3287,37,
65,laojunmiao,CN,39.8333,97.7333,50,
119,zhangye,CN,38.9342,100.4517,56,
175,prince george,CA,53.9166,-122.7530,73,
525,fort mcmurray,CA,56.7268,-111.3810,79,
531,jinchang,CN,38.4953,102.1739,62,
557,chibougamau,CA,49.9168,-74.3659,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [69]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
palmer - nearest hotel: No hotel found
laojunmiao - nearest hotel: No hotel found
zhangye - nearest hotel: Zhangye Hotel 张掖饭店
prince george - nearest hotel: Travelodge by Wyndham Prince George
fort mcmurray - nearest hotel: Nomad Hotel and Suites
jinchang - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
20,palmer,US,42.1584,-72.3287,37,No hotel found
65,laojunmiao,CN,39.8333,97.7333,50,No hotel found
119,zhangye,CN,38.9342,100.4517,56,Zhangye Hotel 张掖饭店
175,prince george,CA,53.9166,-122.7530,73,Travelodge by Wyndham Prince George
525,fort mcmurray,CA,56.7268,-111.3810,79,Nomad Hotel and Suites
531,jinchang,CN,38.4953,102.1739,62,No hotel found
557,chibougamau,CA,49.9168,-74.3659,74,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [72]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   frame_width = 800,
                                   frame_height = 600,                                 
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)